<a href="https://colab.research.google.com/github/motorlearner/neuromatch/blob/main/plot_responsedistributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf getdata.py* sample_data data01_direction4priors.csv
!wget -q https://raw.githubusercontent.com/motorlearner/neuromatch/refs/heads/main/getdata.py
%run getdata.py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# make df visible
df = df

# plot function
def plot_resp(subject_id:int, bin_width:int):
  dat = df[(df.subject_id == subject_id)]
  # sets
  stim_set = np.sort(dat.stim_rel.unique())
  prsd_set = np.sort(dat.prior_sd.unique())[::-1]
  # grid size
  ncols = len(prsd_set)
  nrows = len(stim_set)
  # plot
  fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*2, nrows*1), squeeze=False, sharex=True)
  fig.subplots_adjust(wspace=0.5, hspace=0.1)
  for i, stim_rel in enumerate(stim_set):
    for j, prior_sd in enumerate(prsd_set):
      ax = axes[i,j]
      ax.set_box_aspect(0.5)
      thisdat = dat[(dat.stim_rel == stim_rel) & (dat.prior_sd == prior_sd)]
      if len(thisdat)==0:
        ax.set_visible(False)
      else:
        for spine in ['top', 'right', 'left']:
          ax.spines[spine].set_visible(False)
        # axes
        ax.xaxis.set_ticks_position('bottom')
        ax.tick_params(axis='x', labelsize=8)
        ax.tick_params(axis='y', left=False, labelleft=False)
        ax.set_xlim(-180, 180)
        ax.set_xticks([-180, -90, 0, 90, 180])
        ax.set_xticklabels(['−180°', '−90°', '0°', '90°', '180°'], fontsize=7)
        # data
        ax.axvline(x=stim_rel, lw=1, ls="-", color="navy")
        ax.hist(thisdat.resp_rel, bins=np.arange(-180,181,bin_width), color=colormap[prior_sd])
      if i==0 and j==0:
        ax.annotate(
          f'Subject {subject_id}, Bin width = {bin_width}',
          xy=(1.1, 1), xycoords='axes fraction', va='top', ha='left',
          rotation=0, fontsize=9, fontweight='bold'
        )

# interactive
subject_ids = sorted(df['subject_id'].unique())
bin_options = [i for i in range(1, 20) if 360 % i == 0]

dropdown = widgets.Dropdown(
  options=subject_ids,
  value=subject_ids[0],
  description='Subject: '
)
dropdown_bins = widgets.Dropdown(
  options=bin_options,
  value=5,
  description='Bin Width: '
)

if True:
  # components
  plotoutput = widgets.interactive_output(plot_resp, {'subject_id': dropdown, 'bin_width': dropdown_bins})
  controls   = widgets.HBox([dropdown, dropdown_bins])
  container  = widgets.VBox([controls, plotoutput])
  # layout
  dropdown.layout      = widgets.Layout(margin='0 20px 0 0', width='150px')
  dropdown_bins.layout = widgets.Layout(margin='0 20px 0 0')
  controls.layout      = widgets.Layout(margin='0 0 20px 0')
  # show
  display(container)